In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [2]:
Train_Dir = 'C:/Users/G15/Desktop/Projects/Interview Grading/images/train'
Test_Dir = 'C:/Users/G15/Desktop/Projects/Interview Grading/images/test'

In [3]:
def createDataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths, labels

In [4]:
train = pd.DataFrame()
train['image'],train['label'] = createDataframe(Train_Dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
print(train['image'])

0        C:/Users/G15/Desktop/Projects/Interview Gradin...
1        C:/Users/G15/Desktop/Projects/Interview Gradin...
2        C:/Users/G15/Desktop/Projects/Interview Gradin...
3        C:/Users/G15/Desktop/Projects/Interview Gradin...
4        C:/Users/G15/Desktop/Projects/Interview Gradin...
                               ...                        
28816    C:/Users/G15/Desktop/Projects/Interview Gradin...
28817    C:/Users/G15/Desktop/Projects/Interview Gradin...
28818    C:/Users/G15/Desktop/Projects/Interview Gradin...
28819    C:/Users/G15/Desktop/Projects/Interview Gradin...
28820    C:/Users/G15/Desktop/Projects/Interview Gradin...
Name: image, Length: 28821, dtype: object


In [6]:
print(train)

                                                   image     label
0      C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
1      C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
2      C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
3      C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
4      C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
...                                                  ...       ...
28816  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
28817  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
28818  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
28819  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
28820  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise

[28821 rows x 2 columns]


In [7]:
test = pd.DataFrame()

In [8]:
test['image'],test['label'] = createDataframe(Test_Dir)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [9]:
print(test)

                                                  image     label
0     C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
1     C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
2     C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
3     C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
4     C:/Users/G15/Desktop/Projects/Interview Gradin...     angry
...                                                 ...       ...
7061  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
7062  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
7063  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
7064  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise
7065  C:/Users/G15/Desktop/Projects/Interview Gradin...  surprise

[7066 rows x 2 columns]


In [10]:
from tqdm.notebook import tqdm

In [11]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [12]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\G15\AppData\Roaming\Python\Python312\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [13]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [14]:
x_train = train_features/255.0
x_test = test_features/255.0

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [17]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [18]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [19]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3,3), activation='relu',input_shape = (48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))

model.add(Dense(7, activation = 'softmax'))

C:\Users\G15\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ["accuracy"])

In [22]:
model.fit(x = x_train, y = y_train , batch_size = 128 , epochs = 100 , validation_data = (x_test,y_test))

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 80s 350ms/step - accuracy: 0.2410 - loss: 1.8355 - val_accuracy: 0.2559 - val_loss: 1.7713
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 84s 372ms/step - accuracy: 0.2914 - loss: 1.7345 - val_accuracy: 0.4164 - val_loss: 1.5334
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 81s 357ms/step - accuracy: 0.3972 - loss: 1.5421 - val_accuracy: 0.4643 - val_loss: 1.3865
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 83s 368ms/step - accuracy: 0.4449 - loss: 1.4442 - val_accuracy: 0.5031 - val_loss: 1.3303
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 81s 357ms/step - accuracy: 0.4731 - loss: 1.3780 - val_accuracy: 0.5204 - val_loss: 1.2643
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 82s 362ms/step - accuracy: 0.4882 - loss: 1.3304 - val_accuracy: 0.5313 - val_loss: 1.2339
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 84s 373ms/step - accuracy: 0.5096 - loss: 1.2791 - val_accuracy: 0.5519 - val_loss: 1.2045
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 84s 371ms/step - accuracy: 0.5245 -

In [24]:
model_json = model.to_json()
with open("emotiondetector.json","w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [25]:
from keras.models import model_from_json

In [26]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [27]:
label = ["angry","disgust","fear","happy","neutral","sad","surprise"]

In [28]:
def ef(image):
    img = load_img(image,grayscale = True)
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [31]:
image = 'images/train/angry/229.jpg'
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("Model Prediction : ",pred_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Model Prediction :  angry
